In [ ]:
import datacube
import xarray as xr
import rioxarray
import dask
from dask.distributed import performance_report
from dask.distributed import Client

from datacube.utils import masking
## https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py

%matplotlib inline
import hvplot.xarray

## Connect to ODC
dc = datacube.Datacube()

## Start local Dask cluster
## https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster
n_workers = 1
n_threads = 24
client = Client(processes=False, 
                n_workers=n_workers, 
                threads_per_worker=n_threads)

In [ ]:
"""
Good subset for testing: 
x=(11.5, 11.8)
y=(50.6, 51)
"""

## Define some parameters as variables (so filenames don't need to be changed manually) 
orbit = "asc"
chunk_size = 1500

ds = dc.load(product=f"s1_ARD_{orbit}",
             dask_chunks={'time':-1, 'y':chunk_size, 'x':chunk_size})

ds.VH

In [ ]:
count = masking.valid_data_mask(ds.VH).sum(dim="time")

In [ ]:
nodata = ds.VH.attrs['nodata']
count = xr.where(ds.VH > nodata, True, False).sum(dim="time")
count

In [ ]:
%%time
with performance_report(filename=f"./reports/obs_perpixel_SAR_{orbit}__{n_workers}-{n_threads}_{chunk_x}-{chunk_y}_test.html"):
    count = count.compute()

In [ ]:
count.rio.to_raster(f"./output/obs_perpixel_SAR_{orbit}__{n_workers}-{n_threads}_{chunk_x}-{chunk_y}_test.tif", dtype="float32")